# Stream the raw measurements and plot them

In [ ]:
import serial
import struct
import numpy as np
serial_port = '/dev/ttyACM0'
ADC_BUFFER_LENGTH = 4048
ADC_BUFFER_LENGTH = 1

In [ ]:
def cal_fft(values):
    values = np.array(values, dtype = np.float)
    values = (values - 32767) * (3.3 / 65535.0)
    return np.absolute(np.fft.rfft(values))[1:]

def get_freqs():
    sampling_frequency = 1000
    signal_length = ADC_BUFFER_LENGTH
    return np.arange(ADC_BUFFER_LENGTH/2) * sampling_frequency / signal_length

In [ ]:
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.models.tools import HoverTool
from bokeh.io import output_notebook, show, push_notebook
output_notebook()
# output_file("lines.html")

hTool = HoverTool(
    tooltips = [
        ("frequency", "@time"),
        ("value", "@value"),
    ],
    mode='vline'
)

In [ ]:
from threading import Thread

stop_threads = False

# clean serial port input buffer (ignore previous data)
with serial.Serial(serial_port, 115200) as ser:
    ser.flushInput()
    ser.write(1)
    
my_figure = figure(plot_width=900, plot_height=500)
my_figure.y_range=Range1d(0, (2**16)-1)
my_figure.add_tools(hTool)
source = ColumnDataSource(data=dict(time=[], value=[]))
line = my_figure.line(x='time' , y='value', source = source, legend_label="Temp.", line_width=2)

handle = show(my_figure, notebook_handle=True)

def blocking_callback(id, stop):
    new_data=dict(time=[], value=[])

    step = 0
    step_size = 0.1  # increment for increasing step
    period = .001  # in seconds (simulate waiting for new data)
    n_show = ADC_BUFFER_LENGTH * 5  # number of points to keep and show
    last_timestamp = 0
    freqs = get_freqs()
    
    with serial.Serial(serial_port, 115200) as ser:
        while True:
            byts = ser.read(ADC_BUFFER_LENGTH * 2)
            
            if stop():
                ser.write(0)
                print("exit")
                break

            values = struct.unpack('{}H'.format(ADC_BUFFER_LENGTH), byts)
            
            timestamp = np.linspace(last_timestamp, last_timestamp + (period*ADC_BUFFER_LENGTH), ADC_BUFFER_LENGTH, endpoint=False)
            last_timestamp = timestamp[-1]+period
            
#             fft = cal_fft(values)
#             source.data = dict(
#                 time = freqs,
#                 value = fft
#             )
            new_data['time'] = timestamp
            new_data['value'] = values
            step = step + step_size
            source.stream(new_data, n_show)
            
            push_notebook(handle=handle)
            
            

thread = Thread(target=blocking_callback, args=(id, lambda: stop_threads))
thread.start()

In [ ]:
# you might also want to stop the thread
stop_threads=True
del thread